# Data Science I
### Klausur II im Sommersemester 2024

## Allgemeine Informationen

* Sie haben eine Woche Zeit, um die Klausur zu bearbeiten.

* Sie können alle Quellen verwenden, müssen sie jedoch korrekt benennen. Wenn Sie ChatGPT oder eine ähnliche Software verwenden, müssen Sie dies kenntlich machen und den verwendeten Prompt angeben.

* Sie dürfen in keiner Form Hilfe von Dritten in Anspruch nehmen.

* Sie dürfen Ihren Code nicht mit Dritten teilen (bspw. über ein öffentliches GitHub-Repository)

* Sie sollten die folgenden Pakete verwenden: `numpy, pandas, geopandas, scipy, statsmodels, scikit-learn/sklearn, matplotlib, seaborn` und Pythons Standardlibraries. Diese sind ausreichend, um die Klausur zu lösen. Falls Sie andere Pakete verwenden, rechtfertigen Sie deren Verwendung.

* Falls nicht anders angegeben, nutzen Sie ausschließlich Python Code innerhalb dieses Notebooks zur Lösung der Aufgaben.

* Der Code muss ausreichend kommentiert und verständlich sein. Schreiben Sie Funktionen beim Wiederverwenden von Code. Befolgen Sie im Allgemeinen die Richtlinien aus der Vorlesung. Punkte können aufgrund eines schlecht strukturierten oder unverständlichen Codes abgezogen werden.

* **Begründen Sie immer Entscheidungen** zur Auswahl von Plots, Hypothesentest usw. und **interpretieren Sie** Ihre Ergebnisse.

* Bitte laden Sie Ihr Notebook mit Zellenausgabe sowie alle weiteren Dateien als `.zip`-Verzeichnis mit dem Dateinamen `nachname_matrikelnummer.zip` bis 14. Oktober 2024, 12:00 Uhr mittags auf StudIP in den Ordner `Submission - Exam 2` hoch.

* Fügen Sie dem `.zip`-Verzeichnis auch die unterschriebene Eigenständigkeitserklärung hinzu.

* Wenn Sie Fragen haben, kontaktieren Sie uns bitte rechtzeitig über Rocketchat.

* Nutzen Sie Rocketchat während der Bearbeitungszeit bitte **ausschließlich für Ihre Fragen**.

In [4]:
# IMPORT LIBRARIES
import numpy as np
import pandas as pd
import geopandas as gpd
import scipy.stats as stats
import statsmodels.api as sm
import sklearn
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# Set seaborn as the default for visualizations
sns.set_theme()

## Aufgaben und Punkte:

<table>
  <thead>
    <tr>
      <th colspan="2">Aufgabe 1 - Data Preprocessing</th>
      <th colspan="3">Aufgabe 2 - Plotting</th>
      <th colspan="2">Aufgabe 3 - Statistics</th>
      <th colspan="2">Aufgabe 4 - Machine Learning </th>
    </tr>
    <tr>
      <th>Aufgabe 1.1</th>
      <th>Aufgabe 1.2</th>
      <th>Aufgabe 2.1</th>
      <th>Aufgabe 2.2</th>
      <th>Aufgabe 2.3</th>
      <th>Aufgabe 3.1</th>
      <th>Aufgabe 3.2</th>
      <th>Aufgabe 4.1</th>
      <th>Aufgabe 4.2</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>6 Punkte </td>
      <td>4 Punkte </td>
      <td>13 Punkte </td>
      <td>10 Punkte</td>
      <td>9 Punkte </td>
      <td>11 Punkte </td>
      <td>5 Punkte </td>
      <td>21 Punkte </td>
      <td>21 Punkte </td>
    </tr>
    <!-- Add more rows as needed -->
  </tbody>
</table>


_____
## Aufgabe 0: Setup

Der Klausurordner enthält ein `Dockerfile`, in dem alle relevanten Pakete definiert sind. Das `Dockerfile` baut auf dem Jupyter Server Image auf. Verwenden Sie dieses Dockerfile, um zuerst ein Docker Image zu erstellen und dann einen Docker Container von diesem Image zu starten. Benutzen Sie anschließend die Jupyter Server Instanz, um an der Klausur zu arbeiten. Wir empfehlen dringend, die Docker-Umgebung zu verwenden, um Versionskonflikte zwischen den verschiedenen Paketen zu vermeiden. Code, der in dieser Umgebung nicht ausführbar ist, wird als **nicht funktional** bewertet.

____
## Aufgabe 1: Data Preprocessing (10 Punkte)

### Datenbeschreibung

In dieser Klausur analysieren wir den Datensatz [What A Waste](https://datacatalog.worldbank.org/search/dataset/0039597/What-a-Waste-Global-Database) aus der Datenbank der World Bank. 
Die für uns relevanten Spalten sind u.a.:
- `total_msw_total_msw_generated_tons_year`: Gesamte Tonnen an festen Kommunalabfällen (ff. Abfall)
- `composition_*`: Anteil der jeweiligen Abfallkategorie an `total_msw_total_msw_generated_tons_year`
- `waste_treatment_*`: Weiterverarbeitungsweise des Abfalls
- `waste_collection_*`: Geografische Abdeckung der Abfallabfuhr
- `gpd`: Gross Domestic Product (GDP)/ Bruttoinlandsprodukt (BIP) pro Kopf
- `population_population_number_of_people`: Gesamtbevölkerung

#### Aufgabe 1.1 - Laden der Daten (6 Punkte)
Laden Sie den Datensatz `country_level_data.csv`. Führen Sie anschließend die folgenden Schritte aus:
- Entfernen Sie alle Zeilen mit `nan`-Werten in der Spalte `total_msw_total_msw_generated_tons_year`.
- Fügen Sie dem Dataframe eine neue Spalte `gdp_total` hinzu, die das gesamte GDP für jedes Land angibt.
- Fügen Sie dem Dataframe zwei neue Spalten `relative_waste_pop` und` relative_waste_gdp` hinzu, die die Gesamtabfallmenge relativ zur Gesamtbevölkerung bzw. zum GDP des Landes angibt.

Laden Sie außerdem das Shapefile `data/world-administrative-boundaries/world-administrative-boundaries.shp`. Verwenden Sie dazu den Befehl `geopandas.read_file(<pfad>)`. Diese Datei enthält allgemeine Länderinformationen und -grenzen, die wir später für die Darstellung verwenden werden.

**Hinweis**: `geopandas.GeoDataFrame`-Objekte sind` pandas.DataFrame`-Objekte, die lediglich eine zusätzliche Spalte mit Geometrieinformationen enthalten. Sie können entsprechend alle `pandas`-Methoden mit diesen Dataframes verwenden.

In [7]:
data_path = 'data/country_level_data.csv'
shapefile_path = 'data/world-administrative-boundaries/world-administrative-boundaries.shp'

# Laden des country_level_data.csv DataFrames
df = pd.read_csv(data_path)

# Entfernen von Zeilen mit NaN-Werten in der Spalte 'total_msw_total_msw_generated_tons_year'
df = df.dropna(subset=['total_msw_total_msw_generated_tons_year'])

# Ensure the column name 'gdp' is correctly spelled and exists in the dataframe
df['gdp_total'] = df['gdp'] * df['population_population_number_of_people']

# Hinzufügen der Spalten 'relative_waste_pop' und 'relative_waste_gdp'
df['relative_waste_pop'] = df['total_msw_total_msw_generated_tons_year'] / df['population_population_number_of_people']
df['relative_waste_gdp'] = df['total_msw_total_msw_generated_tons_year'] / df['gdp_total']

# Laden des Shapefiles mit geopandas
gdf = gpd.read_file(shapefile_path)

#### Aufgabe 1.2 - Zusammenführen und Formatieren (4 Punkte)
Fügen Sie die Spalte `continent` aus dem Shapefile-Datenframe zu Ihrem Dataframe der Abfallproduktion mithilfe der Spalten `iso3` und `iso3c` hinzu. Löschen Sie alle Länder für die es keine Übereinstimmung in den beiden Spalten gibt.

_____
## Aufgabe 2: Plotting (32 Punkte)



### Aufgabe 2.1 - Globaler Überblick (13 Punkte)
Erstellen Sie zunächst einen globalen Überblick über die Verteilung von Abfällen, Bevölkerung und Wirtschaftskraft.

#### Aufgabe 2.1.1 (6 Punkte)
Berechnen Sie dazu den Anteil jedes Kontinents an den gesamten globalen Abfällen, der Weltbevölkerung und der Weltwirtschaftsleistung und visualisieren Sie diese in geeigneten Diagrammen.

Der Datensatz enthält die Spalte `income_id`, die das Land als niedriges Einkommen (LIC), niedrigeres mittleres Einkommen (LMC), oberes mittleres Einkommen (UMC) oder hohes Einkommen (HIC) klassifiziert.
Verwenden Sie anstelle des Kontinents die `income_id`, um die Verteilung von Abfall, Bevölkerung und GDP für die Einkommenskategorien zu visualisieren.

Beschreiben Sie Ihre Diagramme und gehen Sie auf die Unterschiede ein.

#### Aufgabe 2.1.2 (7 Punkte)
Schauen Sie sich nun die Abfallproduktion für jedes Land genauer an.<br>
Verwenden Sie dazu den Shapefile-Dataframe und `geopandas` ([siehe hier](https://geopandas.org/en/stable/docs/user_guide/mapping.html)), um drei Karten der Welt in einer Figure darzustellen. Die Farbe der Landesfläche sollte dabei von (1) `total_msw_total_msw_generated_tons_year`, (2) `relative_waste_pop` und (3) `relative_waste_gdp` abhängen. Verwenden Sie logarithmierte Werte für die Farbdarstellung und fügen Sie eine Farbskala hinzu.

Welche Beobachtungen machen Sie? Welche Länder stechen bei Ihrer Darstellung hervor?

### Aufgabe 2.2 - Treiber der Abfallproduktion (10 Punkte)
Betrachten wir nun mögliche Ursachen für die Abfallproduktion.

#### Aufgabe 2.2.1 (6 Punkte)
Visualisieren Sie das Verhältnis vom Abfallaufkommen und GDP, sowie vom Abfallaufkommen und der Gesamtbevölkerung der Länder in zwei geeigneten Plots. Fügen Sie Ihren Plots auch den Korrelationskoeffizienten hinzu.

Welche Variable würden Sie eher verwenden, um das Abfallaufkommen eines Landes mit bekannter Gesamtbevölkerung und GDP vorherzusagen?

#### Aufgabe 2.2.2 (4 Punkte)
Verwenden Sie dieselben Plots wie in Aufgabe 2.2.1, aber log-transformieren Sie alle Variablen. Beschreiben Sie die Unterschiede.

### Aufgabe 2.3 - Abfallaufkommen pro Kopf (9 Punkte)

Betrachten Sie nun die Pro-Kopf-Abfallproduktion.
Länder mit einem hohen Lebensstandard werden oft dafür kritisiert, dass sie große Mengen an organischen Abfällen produzieren, zu denen in der Regel auch essbare Lebensmittel gehören. 

Visualisieren Sie die Abfälle und das BIP pro Kopf zusammen mit der absoluten Menge der organischen Abfälle pro Kopf. Verwenden Sie die Spalte `composition_food_organic_waste_percent`, um die absolute Menge an organischen Abfällen zu berechnen. 
Verwenden Sie eine Scatterplot und wählen Sie die Größe und Farbe der Punkte in Abhängigkeit von der absoluten Menge der organischen Abfälle pro Kopf. 
Beschriften Sie außerdem die 20 Länder mit dem höchsten und die 20 Länder mit dem niedrigsten Pro-Kopf-Aufkommen an organischen Abfällen mit dem Ländernamen und der Position in der Rangliste der organischen Abfälle aller Länder. Beschränken Sie die Darstellung auf Länder mit mindestens 10.000.000 Einwohnern und schließen Sie alle Länder ohne Informationen über den Anteil organischer Abfälle aus.

Welchen Trend können Sie beobachten? 
Welche Länder sind Ausreißer und folgen nicht dem allgemeinen Trend?

___
## Aufgabe 3: Statistics (16 Punkte)


#### Aufgabe 3.1 - Statistischer Hypothesentest (11 Punkte)
In Aufgabe 2.1 haben wir gesehen, dass die HIC- und UMC-Länder den größten Teil des weltweiten Abfalls produzieren und ~50% der Weltbevölkerung ausmachen, d.h. sie können als die einkommensstarke Hälfte der Weltbevölkerung angesehen werden. 

Aber gibt es einen signifikanten Unterschied in der pro Kopf produzierten Abfallmenge zwischen der einkommensstarken und der einkommensschwachen Hälfte der Weltbevölkerung? 

Schreiben Sie das entsprechenden Hypothesenpaar auf und führen Sie einen geeigneten t-Test durch.

#### Aufgabe 3.2 - Statistisches Verständnis (5 Punkte)
Visualisieren Sie die beiden Stichproben in einem Diagramm mithilfe von zwei Histogrammen und beschreiben Sie deren Überlappung.
Finden Sie ein geeignetes Maß, das angibt, wie relevant (**nicht** wie statistisch signifikant) der Unterschied zwischen den beiden Stichproben ist, und interpretieren Sie es.

___
## Aufgabe 4: Machine Learning (42 Punkte)

### Aufgabe 4.1 - Lineare Regression (21 points)

In Aufgabe 2 haben wir die lineare Beziehung zwischen der Gesamtabfallmenge und der Bevölkerung, sowie dem GDP der Länder dargestellt. In dieser Aufgabe wollen wir diese Effekte mithilfe einer linearen Regression weiter untersuchen. Verwenden Sie das Paket `statsmodels` für Ihre Regressionsanalyse.

#### Aufgabe 4.1.1 – GDP (8 Punkte)
Verwenden Sie das GDP als erklärende Variable für die Gesamtabfallmenge eines Landes. Interpretieren Sie die Koeffizienten und die statistische Signifikanz des Effekts. Plotten Sie Ihre Regressionsgerade mit der Gesamtabfallmenge auf der Y-Achse und dem GDP auf der X-Achse. <br>

#### Aufgabe 4.1.2 – Gesamtbevölkerung (8 Punkte)
Verwenden Sie die **log**-Gesamtbevölkerung als erklärende Variable für die **log**-Gesamtabfallmenge eines Landes. Interpretieren Sie den Koeffizienten für die log-Gesamtbevölkerung und plotten Sie Ihre Regressionsgerade mit dem Gesamtabfall auf der Y-Achse und der **log-Gesamtbevölkerung** auf der X-Achse.<br>

#### Aufgabe 4.1.3 – Kritische Einordnung (5 Punkte)
Diskutieren Sie kurz und kritisch folgende Aussage:

*„Die Abfallproduktion eines Landes hängt hauptsächlich von seiner Wirtschaftskraft ab“*

Vergleichen Sie dazu Ihre beiden linearen Regressionen mit einer geeigneten Metrik. 
Ermitteln Sie außerdem den Korrelationskoeffizienten zwischen GDP und Gesamtbevölkerung und beziehen Sie ihn in Ihre Antwort ein.

### Aufgabe 4.2 – Dimensionsreduktion und Clustering (21 Punkte)

#### Aufgabe 4.2.1 - PCA (Principal Component Analysis) (8 Punkte):
Berücksichtigen Sie alle Spalten im Datensatz, die im Zusammenhang mit dem Abfallaufkommen stehen (z. B. Spalten beginnend mit "waste", "composition" oder "treatment" und numerischen Werten) und füllen Sie fehlende Werte in diesen Spalten mit dem Spaltenmittelwert auf. Führen Sie eine **PCA** durch und projizieren Sie die Daten auf die ersten beiden Hauptkomponenten (**PC1** und **PC2**). Visualisieren Sie die Länder in einem Plot mit **PC1** und **PC2**.

#### Aufgabe 4.2.2 - K-Means-Clustering (7 Punkte)
Führen Sie ein **K-Means-Clustering** mithilfe der ersten beiden Hauptkomponenten durch. Nutzen Sie eine **Hyperparametersuche** (z. B. die **Ellbogenmethode**), um die optimale Anzahl von Clustern (**k**) zu bestimmen.

#### Aufgabe 4.2.3 - Auswertung und Vergleich (6 Punkte)
Visualisieren Sie nebeneinander die **optimalen k Cluster** und die **Einkommenskategorie** mithilfe der PCA-Projektion. Vergleichen Sie die Clusterähnlichkeit mit der Einkommensklasse, indem Sie einen Matching-Score berechnen. Diesen Matching-Score können Sie berechnen, indem Sie für jedes Länderpaar prüfen, ob diese zum gleichen K-Means-Cluster und zur gleichen Einkommensklasse gehören. Wenn sich ein Länderpaar sowohl im K-Means-Clustering als auch in der Einkommensklasse im selben Cluster befindet, wird dies als Übereinstimmung betrachtet. Mit anderen Worten, der Matching-Score wird als Prozentsatz der Paare berechnet, bei denen beide Länder in beiden Klassifizierungen derselben Klasse angehören:

$$
\text{Matching Score} = \left( \frac{\text{Number of matching pairs}}{\text{Total number of pairs}} \right) \times 100
$$

Dabei sind **Matching Pairs** Länderpaare, die zum selben K-Means-Cluster und derselben Einkommensklasse gehören, und die **Gesamtzahl der Paare** ist die Gesamtzahl aller möglichen Länderpaare im Datensatz.

Diskutieren Sie auf der Grundlage Ihrer Analyse, ob die Einkommensklasse Ihrer Meinung nach das Abfallwirtschaftsprofil eines Landes widerspiegelt.